<a href="https://colab.research.google.com/github/vincentkoc/oxf-loanapplication_agent-grpfull4/blob/main/oxf_loanapplication_agent_grpfull4_main_colabnotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Oxford AI Summit 2025 Group 4 Full code:

*   Shad Bernard
*   Dimple Francis
*   Omer Gunes
*   Onder Vincent Koc
*   Hrishi Kulkarni
*   Bastiyan Rodrigo


#Agent Orchestration Pattern:

Our loan-workflow solution uses the Parallel Execution orchestration pattern by splitting compliance and fraud into two parallel, tool-backed agents, and then combining their outputs in a final decision step. We utilised the Parallel execution pattern in the following key ways:



***Independent agents for orthogonal subtasks***

We defined two specialized agents—compliance_agent (runs PEP/sanctions, identity, affordability, source-of-wealth, phone checks) and fraud_agent (computes a fraud score and flags)—that operate on the same input application but address completely different concerns. Each agent is free to run its full battery of checks without waiting on the other.

***Concurrent invocation via asyncio.gather***



```
compliance_task = Runner.run(compliance_agent, compliance_prompt)
fraud_task      = Runner.run(fraud_agent, fraud_prompt)
results         = await asyncio.gather(compliance_task, fraud_task)
```
Here both agents are spun up at once in separate async tasks, then await-ed together. This reduces end-to-end latency compared to calling one after the other.

***Aggregation step***

 Once both tasks complete, we immediately unpack their outputs into the shared state:

```
state["compliance_result"] = results[0].final_output
state["fraud_result"]      = results[1].final_output
```

That consolidated state then feeds into the downstream decision_agent, which acts as our “aggregator”—merging the two independent analyses into one final decision.

***Final agent for synthesis***

The decision_agent receives both compliance and fraud results together and returns a single structured verdict. That mirrors the classic “parallel-then-merge” flow: run multiples in parallel, then unify.

#Iterations & Improvements:

**Removed decorator for Postcode validator**

Initially, the fraud check was returning the following error:

```
Fraud result: {"fraud_score":1.0,"flags":["An error occurred while trying to perform the fraud detection. Please try again later."]}
```

This is because the uk_postcode_validator function that was being called from the fraud_detection tool was decorated as a function_tool. Removing this decoration allowed uk_postcode_validator to be called successfully as an ordinary python fuction.

**Told agent not to provide any json formatting**

The decision agent was failing with the following error:

```
[Decision Agent] LLM failed: Decision agent output is not a dict.. Using fallback logic.
```

This was caused because the decision agent was wrapping its JSON output in markdown, which then caused json.loads to fail to convert the JSON string into a dict. This was resolved by instructing the decision agent not to include any markdown in its response.

**Fuzzy logic for name checking**

Our initial solution used exact matching for fraudulent name checks. We improved upon this by implementing a fuzzy logic check, which flags up names that are similar to those on the fraudulent_names list, for manual verification.

**Affordability check improvement**

Our initial solution used used a trivial affordability check: pass if loan amount is less than half of income, else fail. We improved upon this, by including a intermediate band, which would require manual approval, and also by setting a minimum income level for loan approvals.


#Upload Requirements

Upload Requirements.txt file

In [54]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving requirements.txt to requirements (4).txt
User uploaded file "requirements (4).txt" with length 68 bytes


#Install Requirements

Install requirements.txt to ensure that all project dependencies are pulled in before they are used.

In [55]:
!pip install -r requirements.txt

#Import OpenAI Key

In [56]:
from google.colab import userdata
import os

openai_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_key

#Core Environment & Dependencies

This opening block sets up the core environment and dependencies for the Hybrid Agentic Loan Workflow: it imports Python’s asyncio for asynchronous orchestration and patches Colab’s event loop with nest_asyncio.apply(), pulls in typing helpers (TypedDict, Optional, List, Literal) for strict data shapes, brings in the Agent SDK (Agent, Runner, function_tool), and loads utility libraries for regex (re), fuzzy matching (rapidfuzz), phone-number parsing (phonenumbers), and JSON handling. Together, these imports and the nest_asyncio call prepare the notebook to run concurrent agents that parse, validate, and process loan applications.

In [57]:


import asyncio, nest_asyncio
from typing_extensions import TypedDict
from typing import Optional, List, Literal
from agents import Agent, Runner, function_tool
import re
from rapidfuzz import fuzz, process
import phonenumbers
from phonenumbers.phonenumberutil import NumberParseException
import json

nest_asyncio.apply()

#Data Schema & Mock Data

This segment establishes all of the data schemas and mock data the agents will use:

It defines AppState (the shared state object that carries the application data, status flags, results, alerts, and metrics through the workflow) and ApplicationData (the exact shape of the incoming loan request). Then it declares TypedDict return-value schemas for each of the tools (PepSanctionResult, IdentityVerificationResult, AffordabilityResult, SourceOfWealthResult, FraudDetectionResult, and SendAlertResult), ensuring type safety and clear contracts. Finally, it provides a hard-coded FRAUDULENT_NAMES list that the fraud detector will use to flag known or suspicious applicants.

In [58]:
# ---------------------------
# Define the shared state
# ---------------------------
class AppState(TypedDict):
    application_data: dict
    status: str
    compliance_result: Optional[dict]
    fraud_result: Optional[dict]
    decision_result: Optional[str]
    decision_reason: Optional[str]
    alerts: List[str]
    alert_summary: Optional[str]
    metrics: dict


# ---------------------------
# ApplicationData TypedDict
# ---------------------------

class ApplicationData(TypedDict):
    full_name: str
    income: int
    loan_amount: int
    document_id: str
    source_of_wealth: str
    postcode: str
    phone_number: str


# ---------------------------
# Tool Output TypedDicts
# ---------------------------

class PepSanctionResult(TypedDict):
    status: Literal["pass", "fail", "manual_verification"]
    reason: Optional[str]

class IdentityVerificationResult(TypedDict):
    status: Literal["pass", "fail"]
    reason: Optional[str]

class AffordabilityResult(TypedDict):
    status: Literal["pass", "fail", "manual_verification"]
    reason: Optional[str]

class SourceOfWealthResult(TypedDict):
    status: Literal["pass", "fail"]
    reason: Optional[str]

class FraudDetectionResult(TypedDict):
    fraud_score: float
    flags: List[str]

class SendAlertResult(TypedDict):
    status: str

# ---------------------------
# Mock Data for Fraud detection
# ---------------------------

FRAUDULENT_NAMES = [
    "john shady",
    "anna dodgy",
    "mr sanction",
    "lisa scammer",
    "peter fraudwell",
    "olga blacklist",
    "ivan launder",
    "maria suspect",
    "tony riskman",
    "sarah shell",
    "viktor mule",
    "nina fakeid",
    "george bribe",
    "lucy offshor",
    "mohammed sanction",
    "jane alias",
    "david ghost",
    "emily shadow",
    "frankie wire",
    "sophia mule"
]

#Postcode Validator

This helper function performs a simple format check on UK postcodes: it upper-cases the input, then uses a regular expression to enforce the basic structure (one or two letters, a digit or “R”, an optional alphanumeric, a space, a digit, and two letters). It returns True if the postcode matches the pattern, or False otherwise—serving as a quick, mock validation step in the pipeline.

In [59]:
def uk_postcode_validator(postcode: str) -> bool:
    """Validates a UK postcode format. This is a mock implementation."""
    # A very basic regex for UK postcodes.
    pattern = re.compile(r"^[A-Z]{1,2}[0-9R][0-9A-Z]? [0-9][A-Z]{2}$")
    return bool(pattern.match(postcode.upper()))

#Politically Exposed Persons Check Tool

This pep_sanction_check tool wraps a sanctions‐list lookup (decorated for Agent use). It lowercases the input name and first does a direct membership check against the mock FRAUDULENT_NAMES list—returning a "fail" if there’s an exact match. If there’s no exact hit, it then runs a fuzzy string similarity (rapidfuzz) against the same list; if the best match scores above 80%, it returns "manual_verification" with the matching name and score. If neither check flags the name, it returns "pass", indicating the applicant isn’t on—or close to—any watchlist entries.

In [60]:
@function_tool(strict_mode=False)
def pep_sanction_check(name: str) -> PepSanctionResult:
    """
    Checks if a name is on a Political Exposed Person (PEP) or sanctions list, with fuzzy matching.
    Returns 'fail' for exact, 'manual_verification' for close matches, 'pass' otherwise.
    """
    print(f"   - Checking PEP/sanctions for: {name}")
    name_lower = name.lower()
    # Exact match
    if name_lower in FRAUDULENT_NAMES:
        return {"status": "fail", "reason": "Name found on a watchlist (exact match)."}

    # Fuzzy match
    best_match = None
    score = 0
    try:
        best_match, score, _ = process.extractOne(name_lower, FRAUDULENT_NAMES, scorer=fuzz.ratio)
    except Exception:
        pass
    if best_match and score > 80:
        return {"status": "manual_verification", "reason": f"Name similar to watchlist entry: '{best_match}' (score: {score})"}

    return {"status": "pass", "reason": None}

#Identity Verification Check Tool

This identity_verification function (exposed as an Agent tool via @function_tool) mocks an identity‐document check by scanning the provided document content string: it prints a verification message, treats any content containing the substring "invalid" as a failure (returning {"status": "fail", "reason": "Document is invalid."}), and otherwise returns {"status": "pass", "reason": None}. It’s a stand-in for what would, in production, be a call to a real identity-verification service or library.

In [61]:
@function_tool(strict_mode=False)
def identity_verification(document_content: str) -> IdentityVerificationResult:
    """
    Verifies an identity document. This is a mock implementation.
    The content of the document is passed as a string.
    """
    print(f"   - Verifying identity document...")
    if "invalid" in document_content:
        return {"status": "fail", "reason": "Document is invalid."}
    return {"status": "pass", "reason": None}

#Affordability Check Tool

This assess_affordability tool evaluates whether a requested loan is reasonable given the applicant’s income by computing a simple debt-to-income (DTI) ratio. It first prints a log message, then rejects any non-positive inputs. It calculates dti = loan_amount / income and fails outright if DTI exceeds 0.4 (loan > 40% of income), flags for manual review if DTI sits between 0.3 and 0.4, and also fails if the income itself is below a minimum threshold (here £10,000). Only applications with a DTI ≤ 0.3 and sufficient income return a "pass" status.

In [62]:
@function_tool(strict_mode=False)
def assess_affordability(income: int, loan_amount: int) -> AffordabilityResult:
    """
    Assesses the applicant's loan affordability based on income and loan amount.
    This is a mock implementation with a slightly more sophisticated check.
    """
    print(f"   - Assessing affordability for income {income} and loan {loan_amount}")

    # Basic DTI (Debt-to-Income) ratio check
    if income <= 0 or loan_amount <= 0:
        return {"status": "fail", "reason": "Income and loan amount must be positive numbers."}

    dti = loan_amount / income  # Debt-to-Income ratio

    # Fail if DTI is above 0.4 (i.e., loan is more than 40% of annual income)
    if dti > 0.4:
        return {
            "status": "fail",
            "reason": f"Loan amount is too high relative to income (DTI: {dti:.2f})."
        }
    # Warn if DTI is between 0.3 and 0.4 (manual verification)
    if 0.3 < dti <= 0.4:
        return {
            "status": "manual_verification",
            "reason": f"Loan amount is moderately high relative to income (DTI: {dti:.2f})."
        }
    # Also fail if income is below a minimum threshold (e.g., £10,000)
    if income < 10000:
        return {
            "status": "fail",
            "reason": "Income below minimum threshold for loan consideration."
        }
    return {"status": "pass", "reason": None}

#Source of Wealth Check Tool

This source_of_wealth_check tool (decorated for the compliance agent) mocks a source-of-funds review by printing the provided description and then performing a simple keyword check: if the lowercased text contains “illegal,” it returns a "fail" status with a “Suspicious source of wealth” reason; otherwise it returns "pass". It stands in for what, in production, would be a call to a dedicated source-of-wealth verification service.

In [63]:
@function_tool(strict_mode=False)
def source_of_wealth_check(source_description: str) -> SourceOfWealthResult:
    """
    Verifies the applicant's source of wealth. This is a mock implementation.
    """
    print(f"   - Checking source of wealth: {source_description}")
    if "illegal" in source_description.lower():
        return {"status": "fail", "reason": "Suspicious source of wealth."}
    return {"status": "pass", "reason": None}


#Phone Number Validation Check Tool

This phone_number_validator tool wraps Google’s libphonenumber to verify that a given string is a valid phone number for the specified region (defaulting to “GB”). It attempts to parse the input with phonenumbers.parse(), then returns the result of phonenumbers.is_valid_number(). If parsing throws a NumberParseException, it catches it and returns False, so callers get a simple True/False response indicating validity.

In [64]:
@function_tool(strict_mode=False)
def phone_number_validator(phone_number: str, region: str = "GB") -> bool:
    """
    Validates a phone number using Google's libphonenumber.
    Returns True if valid, False otherwise.
    """
    try:
        parsed = phonenumbers.parse(phone_number, region)
        return phonenumbers.is_valid_number(parsed)
    except NumberParseException:
        return False


#Fraud Detection Check Tool

This fraud_detection tool takes the full ApplicationData dict and runs a couple of quick checks to flag potential fraud: it logs that it’s running, lowercases the applicant’s name and adds a “Name is on a watchlist” flag (and +0.8 to the score) if it exactly matches one of the mock FRAUDULENT_NAMES, then calls the uk_postcode_validator to check the postcode—adding “Invalid UK postcode” and +0.3 to the score if that fails. It returns a fraud_score (capped at 1.0) plus any accumulated flags, serving as a simple, mock fraud‐risk assessment.

In [65]:
@function_tool(strict_mode=False)
def fraud_detection(application: ApplicationData) -> FraudDetectionResult:
    """
    Detects fraud signals in the application data. Mock implementation.
    """
    print(f"   - Running fraud detection...")
    flags = []
    score = 0.0
    name = application["full_name"].lower()
    if name in FRAUDULENT_NAMES:
        flags.append("Name is on a watchlist.")
        score += 0.8
    if not uk_postcode_validator(application["postcode"]):
        flags.append("Invalid UK postcode.")
        score += 0.3

    return {"fraud_score": min(1.0, score), "flags": flags}

#Send Alert Tool

This send_alert tool is a mock notification function decorated for Agent use: it takes a message string, prints out " - Sending alert: {message}" to simulate dispatching (e.g., via email or SMS), and returns a {"status": "sent"} dictionary—standing in for what would, in production, be a real alerting service call.

In [66]:
@function_tool(strict_mode=False)
def send_alert(message: str) -> SendAlertResult:
    """Sends an alert. Mock implementation."""
    # TODO: Replace with real MCP call to an alerting service (e.g., email, SMS).
    print(f"   - Sending alert: {message}")
    return {"status": "sent"}

#Deterministic Application Processor

This application_processor function performs the first, non-AI validation step by examining the AppState’s application_data dict for required fields (full_name, income, loan_amount, document_id, source_of_wealth, postcode). It logs “Step 1: Processing application…,” collects any missing fields into an alert message if they’re empty or absent, updates the state’s alerts and sets state["status"] to "incomplete" when fields are missing—or to "validated" if everything is present—and then prints out the resulting status before returning the updated state.

In [67]:
def application_processor(state: AppState) -> AppState:
    """
    Validates the initial application for completeness.
    This is a deterministic step.
    """
    print("Step 1: Processing application...")
    app = state["application_data"]

    required_fields = ["full_name", "income", "loan_amount", "document_id", "source_of_wealth", "postcode"]
    missing_fields = [field for field in required_fields if not app.get(field)]

    if missing_fields:
        state["alerts"].append(f"Application incomplete. Missing fields: {', '.join(missing_fields)}")
        state["status"] = "incomplete"
    else:
        state["status"] = "validated"
    print(f"Application status: {state['status']}")
    return state

#Compliance Agent

This snippet instantiates the compliance_agent, configuring it as an AI “compliance officer” that uses the defined tools to vet a loan application. It has a name (“ComplianceAgent”), a set of plain-English instructions telling it to run each of the compliance checks (PEP/sanctions, identity, affordability, source-of-wealth, and phone-number validation) and to return a consolidated JSON summary, specify the gpt-4o-mini model, and pass in exactly those five tool functions—so that when the agent is invoked, it knows which checks to perform and how to package their outputs.

In [68]:
compliance_agent = Agent(
    name="ComplianceAgent",
    instructions="""
    You are a compliance officer. Use the provided tools to perform all required compliance checks on the loan application.
    The user will provide the application data as a prompt.
    You must call all relevant tools based on the application data (pep_sanction_check, identity_verification, assess_affordability, source_of_wealth_check, phone_number_validator).
    Consolidate the results from all checks into a single JSON object.
    """,
    model="gpt-4o-mini",
    tools=[
        pep_sanction_check,
        identity_verification,
        assess_affordability,
        source_of_wealth_check,
        phone_number_validator,
    ],
)

#Fraud Agent

This defines the fraud_agent as an AI specialist whose sole purpose is to run the fraud_detection tool on an application and return its results. It’s named "FraudAgent", uses the gpt-4o-mini model, and is given clear instructions to output a plain Python dictionary with a float fraud_score and a list of English flags. If anything goes wrong during execution, it must default to a maximum risk (fraud_score = 1.0) and include an explanatory flag—without wrapping any of that in markdown or extra formatting.

In [69]:
fraud_agent = Agent(
    name="FraudAgent",
    instructions="""
    You are a fraud detection specialist. Use the fraud_detection tool to assess the application for fraud risk.
    Always return a dictionary with keys 'fraud_score' (float) and 'flags' (list of strings).
    If you encounter an error, set 'fraud_score' to 1.0 and add a relevant message to 'flags'.
    Never include markdown, code blocks, or embedded JSON in the 'flags' list. Only return plain English strings.
    """,
    model="gpt-4o-mini",
    tools=[fraud_detection],
)

#Decision Agent

This creates the decision_agent configured to synthesize the outputs of the compliance and fraud checks into a final loan decision. It’s named "DecisionAgent", uses the gpt-4o-mini model, and receives instructions to output a JSON object (without any Markdown fencing) containing two keys—decision (one of "approved", "rejected", or "escalated") and reason (a single-sentence explanation). The logic it’s guided to follow is: escalate when compliance fails but fraud is low, reject when fraud score exceeds 0.7, and approve when all compliance checks pass and fraud risk is low.

In [70]:
decision_agent = Agent(
    name="DecisionAgent",
    instructions="""
    Based on the compliance and fraud results, decide if the loan application should be 'approved', 'rejected' or 'escalated'.
    Provide a clear one-sentence reason for your decision.
    - Escalate if compliance checks have failed but fraud risk is low.
    - Reject if fraud risk is high (score > 0.7).
    - Approve if all checks pass and fraud score is low.
    Return a dictionary with 'decision' and 'reason'.
    **Return your answer as raw JSON only, with no markdown fences or prose.**
    """,
    model="gpt-4o-mini",
)

#Alert Agent

This snippet sets up the alert_agent, an AI-powered notification service: it’s named "AlertAgent", uses the gpt-4o-mini model, and is wired to use the send_alert tool. Its instructions tell it to take any incoming message—whether prompting the user about missing fields, compliance failures, or the final decision—and dispatch it via send_alert in a friendly, professional, and supportive tone, helping the applicant understand the next steps or issues with their loan application.

In [71]:
alert_agent = Agent(
    name="AlertAgent",
    instructions="""
    You are an alerting agent for a loan application service.
    When provided with a message, use the send_alert tool to deliver it to the user.
    Ensure the message is communicated in a friendly, professional, and supportive tone,
    helping the user understand any issues or next steps regarding their loan application.
    """,
    model="gpt-4o-mini",
    tools=[send_alert],
)

# Main Workflow Orchestration

This run_workflow coroutine ties together all the pieces into a full loan‐processing pipeline: it initializes the shared AppState, performs a deterministic completeness check (alerting and exiting early if fields are missing), then concurrently invokes the compliance and fraud agents on the application data. Once those finish, it calls the decision agent—parsing its JSON output (with a built-in fallback to simple rules if parsing or the LLM fails)—stores the final verdict and reason, and finally uses the alert agent to send the outcome to the user. Throughout, it logs each major step so its possible to trace the workflow’s progress in the notebook.

In [72]:
async def run_workflow(application: dict):
    """
    Orchestrates the loan application workflow using OpenAI Agents SDK.
    """
    print("🚀 Starting Hybrid Agentic Workflow...")
    # Initialize state
    state = AppState(
        application_data=application,
        status="new",
        compliance_result=None,
        fraud_result=None,
        decision_result=None,
        decision_reason=None,
        alerts=[],
        alert_summary=None,
        metrics={},
    )

    # 1. Deterministic validation
    state = application_processor(state)

    # 2. Handle incomplete application
    if state["status"] == "incomplete":
        print("\nStep 2: Application incomplete. Sending alert.")
        alert_message = state["alerts"][0]
        result = await Runner.run(alert_agent, f"The application is incomplete. Please inform the user with the following message: {alert_message}")
        state["alert_summary"] = result.final_output
        print("Workflow finished: Application incomplete.")
        return state

    # 3. Parallel Compliance and Fraud checks
    print("\nStep 2: Running Compliance and Fraud agents in parallel...")
    compliance_prompt = f"Please run compliance checks for this application: {json.dumps(state['application_data'])}"
    fraud_prompt = f"Please run a fraud check for this application: {json.dumps(state['application_data'])}"

    # Use asyncio.gather to run agents concurrently
    compliance_task = Runner.run(compliance_agent, compliance_prompt)
    fraud_task = Runner.run(fraud_agent, fraud_prompt)

    results = await asyncio.gather(compliance_task, fraud_task)

    state["compliance_result"] = results[0].final_output
    state["fraud_result"] = results[1].final_output
    print(f"Compliance result: {state['compliance_result']}")
    print(f"Fraud result: {state['fraud_result']}")

    # 4. Decision Agent
    print("\nStep 3: Running Decision agent...")
    decision_prompt = f"""
Please make a decision based on these results:
Compliance: {state['compliance_result']}
Fraud: {state['fraud_result']}
"""
    try:
        result = await Runner.run(decision_agent, decision_prompt)
        decision_output = result.final_output
        # Try to parse string output as JSON if needed
        if isinstance(decision_output, str):
            try:
                decision_output = json.loads(decision_output)
            except Exception:
                #pass
                raise
        if isinstance(decision_output, dict):
            state["decision_result"] = decision_output.get("decision")
            state["decision_reason"] = decision_output.get("reason")
        else:
            raise ValueError("Decision agent output is not a dict.")
    except Exception as e:
        print(f"[Decision Agent] LLM failed: {e}. Using fallback logic.")
        # Fallback deterministic rules
        comp = state["compliance_result"]
        fraud = state["fraud_result"]
        # Ensure comp is a dict before using .values()
        affordability_pass = False
        if isinstance(comp, dict):
            affordability_pass = all(
                (c.get('status') == 'pass')
                for c in comp.values() if isinstance(c, dict)
            )
        # Ensure fraud is a dict before using .get()
        fraud_score = 1.0
        if isinstance(fraud, dict):
            fraud_score = fraud.get("fraud_score", 1.0)
        if affordability_pass and fraud_score < 0.7:
            state["decision_result"] = "approved"
            state["decision_reason"] = "Fallback: Checks passed and fraud risk is below threshold."
        else:
            state["decision_result"] = "rejected"
            state["decision_reason"] = "Fallback: Checks failed or fraud risk is too high."

    print(f"Decision: {state['decision_result']} - Reason: {state['decision_reason']}")

    # 5. Alerting
    print("\nStep 4: Sending final notification alert...")
    alert_message = f"Loan application decision: {state['decision_result']}. Reason: {state['decision_reason']}"
    await Runner.run(alert_agent, alert_message)
    print("\n✅ Workflow finished.")
    return state


#Run Loan Application Check With Dummy Data

This final section defines an async def main() driver that runs four illustrative scenarios—clean, fraudulent, fuzzy-name/manual‐verification, and incomplete applications—calling run_workflow() on each, printing headers and the resulting state for easy inspection. It then provides the standard if __name__ == "__main__": guard, re‐applying nest_asyncio (since Colab or other environments may already have an event loop) and invoking asyncio.run(main()) so the notebook or script actually executes all four end‐to‐end demonstrations when the notebook is launched.

In [73]:
async def main():
    # Example 1: A good application
    print("--- Running Scenario 1: Clean Application ---")
    clean_application = {
        "full_name": "Alice Wonderland",
        "income": 80000,
        "loan_amount": 15000,
        "document_id": "valid_passport.pdf",
        "source_of_wealth": "employment",
        "postcode": "SW1A 0AA",
        "phone_number": "+447911123456"
    }
    final_state_1 = await run_workflow(clean_application)
    print("\n--- FINAL STATE (Scenario 1) ---")
    print(final_state_1)

    print("\n\n" + "="*50 + "\n")

    # Example 2: A fraudulent application
    print("--- Running Scenario 2: Fraudulent Application ---")
    fraudulent_application = {
        "full_name": "John Shady",
        "income": 50000,
        "loan_amount": 25000,
        "document_id": "valid_id.jpg",
        "source_of_wealth": "inheritance",
        "postcode": "B4D C0D3", # Invalid postcode
        "phone_number": "07911123456"  # Invalid format (missing +44)
    }
    final_state_2 = await run_workflow(fraudulent_application)
    print("\n--- FINAL STATE (Scenario 2) ---")
    print(final_state_2)

    print("\n\n" + "="*50 + "\n")

    # Example 3: A similar name (should trigger manual verification)
    print("--- Running Scenario 3: Similar Name (Manual Verification) ---")
    similar_name_application = {
        "full_name": "Jon Shadey",  # Similar to 'john shady'
        "income": 60000,
        "loan_amount": 10000,
        "document_id": "valid_id.jpg",
        "source_of_wealth": "salary",
        "postcode": "EC1A 1BB",
        "phone_number": "+447911654321"
    }
    final_state_3 = await run_workflow(similar_name_application)
    print("\n--- FINAL STATE (Scenario 3) ---")
    print(final_state_3)

    print("\n\n" + "="*50 + "\n")

    # Example 4: Incomplete application (should trigger an alert)
    print("--- Running Scenario 4: Incomplete Application (Missing Fields) ---")
    incomplete_application = {
        "full_name": "Bob Missingfields",
        # "income" is missing
        "loan_amount": 5000,
        # "document_id" is missing
        "source_of_wealth": "savings",
        "postcode": "W1A 1AA",
        "phone_number": "+447911000000"
    }
    final_state_4 = await run_workflow(incomplete_application)
    print("\n--- FINAL STATE (Scenario 4) ---")
    print(final_state_4)


# ---------------------------
# Entry point
# ---------------------------
if __name__ == "__main__":
    # Ensure you have set the OPENAI_API_KEY environment variable
    import nest_asyncio, asyncio
    nest_asyncio.apply()
    asyncio.run(main())



--- Running Scenario 1: Clean Application ---
🚀 Starting Hybrid Agentic Workflow...
Step 1: Processing application...
Application status: validated

Step 2: Running Compliance and Fraud agents in parallel...
   - Running fraud detection...
   - Checking PEP/sanctions for: Alice Wonderland
   - Verifying identity document...
   - Assessing affordability for income 80000 and loan 15000
   - Checking source of wealth: employment
Compliance result: Here are the results of the compliance checks for the loan application:

```json
{
  "pep_sanction_check": {
    "status": "pass",
    "reason": null
  },
  "identity_verification": {
    "status": "pass",
    "reason": null
  },
  "assess_affordability": {
    "status": "pass",
    "reason": null
  },
  "source_of_wealth_check": {
    "status": "pass",
    "reason": null
  },
  "phone_number_validator": {
    "status": "valid",
    "reason": null
  }
}
``` 

All compliance checks have passed successfully.
Fraud result: {"fraud_score":0.0,"flags